In [1]:
from pathlib import Path
import os
import glob

import pandas as pd
import numpy as np

import re

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, interactive_output

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing

In [2]:
def search_word(string, word):
    """Funtion to search for a word (separated by spaces) in a string
            [! case sensitive]"""
    return f' {word} ' in f' {string} '


def idx_of_found_item(data, word:str):
    """ Return the corresponding index for a str in an 'array' which contains the 'word'
    Input: 
        data - array of strings
        word - sequence which you are looking for (the function will look for a sequence separated by spaces)
        (it is case sensitive)
    Output: 
        stored_idx - if only one match was found the index will be stored
    """
    stored_idx = []
    for idx, string in enumerate(data.values):
        if search_word(string, word):
            stored_idx.append(data.index[idx])
    #         print(fst_col.index[idx])
        elif idx == data.size-1 and len(stored_idx)<1:
            stored_idx = ['Sequence not found']
        elif idx == data.size-1 and len(stored_idx)>1:
            stored_idx = ['Be more specific. Many similar sequences found.']
    return stored_idx[0]

In [3]:
# Explore the directory to find excel files

currentDir = Path().absolute() # full path to this code
filelevel = 0 # specify Nº of 'levels up' to find the excel file folder

parts_of_currentDir = currentDir.parts
ExcelFileDir = Path(*parts_of_currentDir[:filelevel])

path = currentDir # or any other path of interest

extension = 'xlsx'
os.chdir(path)
files_list = glob.glob('*.{}'.format(extension))
print(files_list)

['OrganicChemicals.xlsx']


In [4]:
# Process raw data

wkb_names = list(map(lambda st: str.replace(st, '.xlsx', ''), files_list)) # names of the workbooks, to be stored as well

appended_workbooks = []

for idx_file, file in enumerate(files_list):
    
    # Read a workbook
    ExcelFilePath = ExcelFileDir / file
    xl = pd.ExcelFile(ExcelFilePath)
    df = pd.read_excel(xl, sheet_name=None)  # read every sheet of the workbook
    
    lcia_methods_labels = []
    products_per_sheet = []
    appended_data = []
    
    for key in df.keys():
        # Read raw data from the sheet
        df_per_sheet = df.get(key)
        
        # There is one LCIA method per sheet of data, identify and store it.   
        idx_method = idx_of_found_item(df_per_sheet.iloc[:,0], 'Method:')  # find the index of the 0-th column where "keyword" appears
        name_method = df_per_sheet.iloc[idx_method,1] # the name of the method is in [idx_method, 1-st column], store it
        lcia_methods_labels.append(name_method)
        
        # Identify the number of products per sheet. They should be the same...
        # Products are numerated sequentially. Last product is in the cell previous to keyword "Method:"
        num_products = int(re.search(r'\d+', df_per_sheet.iloc[idx_method-1,0]).group()) # extract the digits only
        products_per_sheet.append(num_products)
        
        # In every sheet, identify the row where the relevant data starts
        # Drop all the rows before the specified one
        # Store the data range of interest   
        idx_category = idx_of_found_item(df_per_sheet.iloc[:,0], 'category') # find the index of the 0-th column where "keyword" appears
        name_category = df_per_sheet.iloc[idx_category,0] # store the name of the cell [idx_category,0-th column]
        
        new_df = df_per_sheet.drop(range(idx_category), axis=0)
        new_df.columns = new_df.iloc[0]
        new_df.drop(new_df.index[0], inplace=True)

        new_df.insert(0,column='Method', value=name_method, allow_duplicates=False) # add method name to df
        
        new_df.rename(columns={name_category:'Impact (or Damage) category'}, inplace=True) # rename "category" column
        appended_data.append(new_df)
    df_raw_workbook = pd.concat(appended_data, ignore_index=True)
    
    # Make a df with multiindex
    mltindex = pd.MultiIndex.from_arrays([df_raw_workbook.iloc[:,0],df_raw_workbook.iloc[:,1]])
    df_raw_workbook.set_index(mltindex, inplace=True)
    df_raw_workbook.drop(list(df_raw_workbook.index.names),axis=1, inplace=True)
    
    df_raw_workbook = df_raw_workbook.T

# ------------ IF MULTIINDICES REQUIRED UN-COMMENT THE ROWS BELOW ------------    
#     appended_workbooks.append(df_raw_workbook)
# df_raw = pd.concat(appended_workbooks, keys=wkb_names, names=['Workbook','Market'], ignore_index=False) # creates a multiindex

# # Store df of untis of LCIA methods
# df_units = df_raw.xs('Unit', level=1, axis=0, drop_level=False).iloc[0] # if there are multiindices in the df
# # df_units = df_units_1.drop_duplicates(inplace=False)

# df_raw.drop(labels='Unit', axis=0, level=1, inplace=True) # if there are multiindices in the df
# ------------ UN-COMMENT UP TO THE ROW ABOVE ------------

# ===============================================================================================

# ------------ IF SIMPLE INDICES REQUIRED UN-COMMENT THE ROWS BELOW ------------    
    df_raw_workbook.insert(0, column=('Workbook','(blank)'), value=wkb_names[idx_file], allow_duplicates=False) # insert a column w/ name of workbook
    
    appended_workbooks.append(df_raw_workbook)
df_raw = pd.concat(appended_workbooks, keys=None, names=['Market'], ignore_index=False) # w/o multiindex

# Store df of untis of LCIA methods
df_units = df_raw.xs('Unit', level=None, axis=0, drop_level=False).iloc[0] # w/o multiindices

df_raw.drop(labels='Unit', axis=0, level=None, inplace=True) # w/o multiindices
# ------------ UN-COMMENT UP TO THE ROW ABOVE ------------ 

# Transform the data to numeric values (where possible) NEEDED??
# df_raw = pd.to_numeric(df_raw, )

    
# lcia_methods_labels    
# products_per_sheet  
# df_raw.head(3)

In [5]:
df_raw_temp2 = df_raw.iloc[:,1:] # remove the column with categorical data

# Normalized data with respect to max and min in each column
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df_raw_temp2.values)
df_norm_temp = pd.DataFrame(x_scaled, index=df_raw_temp2.index, columns=df_raw_temp2.columns)

# include the column with categorical data
df_norm = pd.concat([df_raw.iloc[:,0],df_norm_temp], axis=1)
# df_norm.head(3)

In [6]:
# def plot_histogram ()
# df_raw.plot.hist(by=df_raw.columns[1], legend=True)


In [15]:
# Widgets
style = {'description_width': 'initial'}

# TEMPORALLY SET BY HAND
df_data = df_raw
# df_data = df_norm

# Make dictionary from multiindexed columns
cols_dict = {}
for i in df_data.columns.values.tolist():
    cols_dict.setdefault(i[0],[]).append(i[1])

LCIA_method = widgets.Select(description='LCIA method:',
                             style=style,
                             value=list(cols_dict.keys())[1],
                             options=list(cols_dict.keys())[1:],
                             layout=widgets.Layout(height='130px', width='100%') #,display='inline-flex',flex_flow='row wrap')
                            )

selected_method = LCIA_method.value

LCIA_category = widgets.Dropdown(description='Method category:',
                                 style=style,
                                 options=cols_dict[selected_method],
                                 layout=widgets.Layout(width='100%', display='inline-flex') #,flex_flow='row wrap')
                                )

# Make third widget depend on the second
def update_categories(*args):
    LCIA_category.options = cols_dict[LCIA_method.value]

LCIA_method.observe(update_categories, names='value')

# Function to plot histogram and KDE
def plot_histogram(method, category):
    fig = plt.figure(figsize=(8,4))
    
    # define the axis
    left, width = 0.1, 0.8
    bottom, height = 0.1, 0.9

    rect_hist = [left, bottom, width, height]

    ax_hist = plt.axes(rect_hist)
       
    ax_hist.hist(df_data[method][category], 
            bins=70, 
            density=False, 
            histtype='bar', # 'stepfilled'# 'step'
            )
    ax_hist.set_xlabel(category)
    ax_hist.set_ylabel('Frequency')
      
# Print statistical info about the data in the columns    
def stat_data(method, category):
    print('')
    print('Statistic data:')
    print(df_data[method][category].describe())
    
# Display
label_1 = widgets.Label(value='Select LCIA method and category:')
ui = widgets.HBox([LCIA_method, LCIA_category])
out_plot = interactive_output(plot_histogram, {'method': LCIA_method, 'category': LCIA_category})
out_stat = interactive_output(stat_data, {'method': LCIA_method, 'category': LCIA_category})

ui_2 = widgets.HBox([out_plot, out_stat])

display(label_1)
display(ui) 
display(ui_2)

Label(value='Select LCIA method and category:')

In [19]:
def create_axes(title, figsize=(14,5)):
    fig = plt.figure(figsize=figsize)
#     fig = plt.figure(constrained_layout=True, figsize=figsize) # active when interavtie plotting will be available
    fig.suptitle(title)
#     fig.canvas.toolbar_position = 'top'   # disabled because %matplotlib widget needs a fix, check in the import section

    # define the axis for the first plot
    left, width = 0.1, 0.3
    bottom, height = 0.1, 0.65
    bottom_h = height + 0.15
    left_h = left + width + 0.02

    rect_scatter = [left, bottom, width, height]
    rect_histx = [left, bottom_h, width, 0.1]
    rect_histy = [left_h, bottom, 0.05, height]

    ax_scatter = plt.axes(rect_scatter)
    ax_histx = plt.axes(rect_histx)
    ax_histy = plt.axes(rect_histy)

    # define the axis for the zoomed-in plot
    left = width + left + 0.2
    left_h = left + width + 0.02

    rect_scatter = [left, bottom, width, height]
    rect_histx = [left, bottom_h, width, 0.1]
    rect_histy = [left_h, bottom, 0.05, height]

    ax_scatter_zoom = plt.axes(rect_scatter)
    ax_histx_zoom = plt.axes(rect_histx)
    ax_histy_zoom = plt.axes(rect_histy)

#     # define the axis for the colorbar
#     left, width = width + left + 0.13, 0.01

#     rect_colorbar = [left, bottom, width, height]
#     ax_colorbar = plt.axes(rect_colorbar)

    return ((ax_scatter, ax_histy, ax_histx),
            (ax_scatter_zoom, ax_histy_zoom, ax_histx_zoom),
#             ax_colorbar
           )


def plot_distribution(axes, X, Y, hist_nbins=50, title="",
                      x0_label="", x1_label=""):
    ax, hist_X1, hist_X0 = axes

    ax.set_title(title)
    ax.set_xlabel(x0_label)
    ax.set_ylabel(x1_label)

    # The scatter plot
#     colors = cmap(y)
    ax.scatter(X, Y, alpha=0.5, marker='o', s=5, lw=0, c='red') #, c=colors)

    # Removing the top and the right spine for aesthetics
    # make nice axis layout
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))

    # Histogram for axis X1
    hist_X1.set_ylim(ax.get_ylim())
    hist_X1.hist(Y, bins=hist_nbins, orientation='horizontal',
                 color='grey', ec='grey')
    hist_X1.axis('off')

    # Histogram for axis X0
    hist_X0.set_xlim(ax.get_xlim())
    hist_X0.hist(X, bins=hist_nbins, orientation='vertical',
                 color='grey', ec='grey')
    hist_X0.axis('off')

In [20]:
def make_plot(method_1, category_1, method_2, category_2, percentile=99):
    """
    category_1, category_2 = any category of the LCIA methods
    percentile = PErcentile of data to show in "Zoom-in" figure (cut off the outliers)
    """
    
    X = df_data[method_1][np.array(category_1)]
    Y = df_data[method_2][np.array(category_2)]
    
#     ax_zoom_out, ax_zoom_in, ax_colorbar = create_axes(title)
    ax_zoom_out, ax_zoom_in = create_axes(f'{category_1} vs {category_2}')
    axarr = (ax_zoom_out, ax_zoom_in)
    plot_distribution(axarr[0], X, Y, hist_nbins=200,
                      x0_label=category_1,
                      x1_label=category_2,
                      title="Full data"
                     )  
        
    X_and_Y = np.column_stack((X,Y))
    
    # zoom-in
    zoom_in_percentile_range = (0, percentile)
    cutoffs_X0 = np.percentile(X_and_Y[:,0], zoom_in_percentile_range)
    cutoffs_X1 = np.percentile(X_and_Y[:,1], zoom_in_percentile_range)

    non_outliers_mask = (
        np.all(X_and_Y > [cutoffs_X0[0], cutoffs_X1[0]], axis=1) &
        np.all(X_and_Y < [cutoffs_X0[1], cutoffs_X1[1]], axis=1))
    
    
    plot_distribution(axarr[1], X[non_outliers_mask], Y[non_outliers_mask],
                      hist_nbins=50,
                      x0_label=category_1,
                      x1_label=category_2,
                      title="Zoom-in")

In [21]:
# Widgets
style = {'description_width': 'initial'}

# Make dictionary from multiindexed columns
cols_dict = {}
for i in df_data.columns.values.tolist():
    cols_dict.setdefault(i[0],[]).append(i[1])

    
# -----------------------------------------
LCIA_method_3 = widgets.Select(description='LCIA method 1:',
                             style=style,
                             value=list(cols_dict.keys())[1],
                             options=list(cols_dict.keys())[1:],
                             layout=widgets.Layout(height='130px', width='100%') #,display='inline-flex',flex_flow='row wrap')
                            )

selected_method_3 = LCIA_method_3.value

LCIA_category_3 = widgets.Dropdown(description='Method category(ies):',
                                 style=style,
                                 options=cols_dict[selected_method_3],
                                 layout=widgets.Layout(width='100%', display='inline-flex') #,flex_flow='row wrap')
                                )

# Make third widget depend on the second
def update_categories_3(*args):
    LCIA_category_3.options = cols_dict[LCIA_method_3.value]

LCIA_method_3.observe(update_categories_3, names='value')

# -----------------------------------------
LCIA_method_4 = widgets.Select(description='LCIA method 2:',
                             style=style,
                             value=list(cols_dict.keys())[1],
                             options=list(cols_dict.keys())[1:],
                             layout=widgets.Layout(height='130px', width='100%') #,display='inline-flex',flex_flow='row wrap')
                            )

selected_method_4 = LCIA_method_4.value

LCIA_category_4 = widgets.Dropdown(description='Method category(ies):',
                                 style=style,
                                 options=cols_dict[selected_method_4],
                                 layout=widgets.Layout(width='100%', display='inline-flex') #,flex_flow='row wrap')
                                )

# Make third widget depend on the second
def update_categories_4(*args):
    LCIA_category_4.options = cols_dict[LCIA_method_4.value]

LCIA_method_4.observe(update_categories_4, names='value')

# -----------------------------------------

slider = widgets.IntSlider(description='Percentile:', 
                          value=99,
                          min=80, max=100, step=1,
                          continuous_update=False)


# Display
label_3 = widgets.Label(value='Select LCIA methods and categories:')
ui_3 = widgets.HBox([LCIA_method_3, LCIA_category_3])
ui_4 = widgets.HBox([LCIA_method_4, LCIA_category_4])

out_plot = interactive_output(make_plot, {'method_1': LCIA_method_3,
                                          'category_1': LCIA_category_3,
                                          'method_2': LCIA_method_4,
                                          'category_2': LCIA_category_4,
                                          'percentile': slider
                                         })

display(label_3)
display(ui_3)
display(ui_4)
display(slider)
display(out_plot)

Label(value='Select LCIA methods and categories:')

IntSlider(value=99, continuous_update=False, description='Percentile:', min=80)

Output()